In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import re

In [3]:
filepath = 'D:/VCCorp/data/traindatatopic.txt'  

In [4]:
f = open(filepath,encoding='utf8')

In [5]:
str = f.read()

In [6]:
label = re.findall(r'<label>(.*?)</label>', str)
content = re.findall(r'<content>\s*((?:.|\n)*?)</content>', str)
print(len(label))
print(len(content))

4965
4965


In [7]:
import pandas as pd

In [8]:
df = pd.DataFrame()

In [9]:
df['content'] = content
df['label'] = label

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4965 entries, 0 to 4964
Data columns (total 2 columns):
content    4965 non-null object
label      4965 non-null object
dtypes: object(2)
memory usage: 77.7+ KB


In [11]:
labels = df['label'].value_counts()

In [12]:
labels

sentiment           2552
advertisement       1337
other topics         393
purchase             309
recruit              189
foreign language     184
oreign language        1
Name: label, dtype: int64

In [13]:
from sklearn.utils import resample

In [14]:
df_majority = df[df.label=='sentiment']

In [15]:
df_imbalanced1 = df[df.label=='advertisement']

In [16]:
df_imbalanced2 = df[df.label=='other topics']
df_imbalanced3 = df[df.label=='purchase']
df_imbalanced4 = df[df.label=='recruit']
df_imbalanced5 = df[df.label=='foreign language']
df_imbalanced6 = df[df.label=='oreign language']

In [17]:
df_unsampled1 = resample(df_imbalanced1, replace=True, n_samples=2552, random_state=42)

In [18]:
df_unsampled2 = resample(df_imbalanced2, replace=True, n_samples=2552, random_state=42)
df_unsampled3 = resample(df_imbalanced3, replace=True, n_samples=2552, random_state=42)
df_unsampled4 = resample(df_imbalanced4, replace=True, n_samples=2552, random_state=42)
df_unsampled5 = resample(df_imbalanced5, replace=True, n_samples=2552, random_state=42)
df_unsampled6 = resample(df_imbalanced6, replace=True, n_samples=2552, random_state=42)

In [19]:
df_unsampled = pd.concat([df_majority, df_unsampled1])

In [20]:
df_unsampled = pd.concat([df_unsampled, df_unsampled2])
df_unsampled = pd.concat([df_unsampled, df_unsampled3])
df_unsampled = pd.concat([df_unsampled, df_unsampled4])
df_unsampled = pd.concat([df_unsampled, df_unsampled5])
df_unsampled = pd.concat([df_unsampled, df_unsampled6])

In [21]:
df_unsampled.label.value_counts()

advertisement       2552
foreign language    2552
sentiment           2552
recruit             2552
purchase            2552
other topics        2552
oreign language     2552
Name: label, dtype: int64

In [22]:
def split_data(data, test_ratio):
    shuffled_id = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_ids = shuffled_id[:test_set_size]
    train_ids = shuffled_id[test_set_size:]
    return data.iloc[train_ids], data.iloc[test_ids]

In [23]:
train_set, test_set = split_data(df_unsampled, 0.2)

In [24]:
from sklearn.feature_extraction.text import CountVectorizer

In [25]:
from sklearn.feature_extraction.text import TfidfTransformer

In [26]:
from sklearn.linear_model import SGDClassifier

In [27]:
from sklearn.pipeline import Pipeline

In [28]:
clf = Pipeline([
    ('cv', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                         alpha=0.01, random_state=42,
                         max_iter=5, tol=None))
])

In [29]:
clf.fit(train_set.content, train_set.label)

Pipeline(memory=None,
     steps=[('cv', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_a...ty='l2', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False))])

In [30]:
pred = clf.predict(test_set.content)

In [31]:
np.mean(pred==test_set.label)

0.8608622620380739

# random forest

In [50]:
from sklearn.ensemble import RandomForestClassifier

In [51]:
clf1 = Pipeline([
    ('cv', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf1', RandomForestClassifier())
])

In [52]:
clf1.fit(train_set.content, train_set.label)

Pipeline(memory=None,
     steps=[('cv', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_a...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [53]:
pred1 = clf1.predict(test_set.content)

In [54]:
np.mean(pred1 == test_set.label)

0.9686450167973124

In [55]:
from sklearn.model_selection import cross_val_score

In [56]:
train_set1, test_set1 = split_data(train_set, 0.4)

In [57]:
scores = cross_val_score(clf1, train_set1.content, train_set1.label, cv=5)

In [58]:
scores.mean()

0.9428627820189955